# About yield

使用 Copilot 答疑。

## 1 一个函数有一个 for 循环，for 循环内有 yield，这个函数返回什么

这个函数返回的是一个生成器对象。在 Python 中，包含 yield 关键字的函数被称为生成器函数。当你调用这样的函数时，它并不立即执行，而是返回一个生成器对象。你可以通过在这个对象上调用 next() 方法来逐步执行函数中的代码，每次执行到 yield 语句时会暂停并返回 yield 后面的值，等待下一次 next() 调用。

In [ ]:
def my_generator():
    for i in range(3):
        yield i

gen = my_generator()
print(next(gen))  # 输出: 0
print(next(gen))  # 输出: 1
print(next(gen))  # 输出: 2

在这个例子中，my_generator 是一个生成器函数，gen 是一个生成器对象。我们可以通过调用 next(gen) 来逐步获取 yield 的值。当所有的值都被 yield 出来后，再次调用 next(gen) 会抛出 StopIteration 异常，表示生成器已经没有更多的值可以生成了。这就是生成器的基本工作原理。

给出一个迭代生成器对象的例子：

In [ ]:
def count_up_to(n):
    """一个生成器函数，生成从1到n的整数"""
    num = 1
    while num <= n:
        yield num
        num += 1

# 创建一个生成器对象
counter = count_up_to(5)

# 迭代生成器对象
for num in counter:
    print(num)

## 2 这个生成器对象是 apache beam DoFn 的 process 函数的返回值

如果生成器对象是 Apache Beam 的 DoFn 的 process 函数的返回值，那么这个生成器对象会被 Beam 的运行时环境用于产生输出元素。

在 Apache Beam 中，DoFn 是一个处理函数，它定义了如何处理输入元素以及如何生成输出元素。process 方法是 DoFn 的核心，它接收一个输入元素，并可以产生任意数量的输出元素。

**当 process 方法返回一个生成器对象时，Beam 的运行时环境会迭代这个生成器，将每次 yield 出来的值作为一个输出元素**。这样，你可以在 process 方法中使用 yield 来产生多个输出元素。

这是一个简单的例子：

In [ ]:
%pip install --quiet apache-beam
%pip install --quiet pandas

import apache_beam as beam

class MyDoFn(beam.DoFn):
    def process(self, element):
        for i in range(element):
            yield i

with beam.Pipeline() as pipeline:
    output = (
        pipeline
        | beam.Create([3, 4, 5])
        | beam.ParDo(MyDoFn())
        | beam.io.WriteToText("yield_output", file_name_suffix = ".txt")
    )

! head yield_output*.txt

从结果上看，最后产生的输出元素就像一个列表：

是的，你的理解是正确的。虽然生成器对象和列表在使用上有一些相似之处，但它们在内部实现上是不同的。

当你从生成器对象中获取元素时，这些元素是在你请求它们的时候才被计算出来的，而不是像列表那样一开始就全部计算好。这就是为什么生成器对象可以用于处理大规模数据集的原因，因为它们不需要一次性将所有的元素都加载到内存中。

然而，从使用者的角度来看，生成器对象产生的输出元素的确就像一个列表。你可以通过迭代生成器对象来获取元素，就像迭代列表一样。在 Apache Beam 的 DoFn 的 process 函数中，你可以使用 yield 来产生多个输出元素，这些元素会被 Beam 的运行时环境收集起来，就像一个列表一样。

总的来说，虽然生成器对象在内部实现上和列表不同，但在使用上，它们产生的输出元素的确可以被看作是一个列表。

到底产生了几个生成器对象？为何最后的输出却像是几个生成器输出元素的合并？

beam.ParDo(MyDoFn()) 会为输入的每一个元素产生一个生成器对象。因此，对于输入 [3, 4, 5]，beam.ParDo(MyDoFn()) 会产生三个生成器对象。

然而，Apache Beam 的运行时环境并不会将这些生成器对象分别处理，而是将它们的输出元素合并到一起。这就是为什么最后的输出看起来像是几个生成器输出元素的合并。

具体来说，当 Beam 的运行时环境迭代每一个生成器对象时，它会将 yield 出来的值收集起来，然后将这些值作为 ParDo 的输出元素。因此，尽管 ParDo 为每一个输入元素产生了一个生成器对象，但最后的输出看起来就像是一个列表，这个列表包含了所有生成器对象的输出元素。

总的来说，**beam.ParDo(MyDoFn()) 为每一个输入元素产生一个生成器对象，但 Beam 的运行时环境会将这些生成器对象的输出元素合并到一起，形成最后的输出**。